In [12]:
import sys
stdout = sys.stdout
import re, math, nltk, datetime, warnings, csv
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.cm as cm
from cycler import cycler
import seaborn as sns
import itertools
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML
# import plotly.graph_objs as go
# from plotly.offline import init_notebook_mode,iplot
sys.stdout = stdout
# init_notebook_mode(connected=True)
# warnings.filterwarnings("ignore")
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline


# configure pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

In [2]:
# load wrangled retail df
ret= pd.read_csv('/home/jk/repos/predictive-segmentation/data/online_retail_II_wrangled.csv')
print(ret.shape)
ret.head()

(417534, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom


In [23]:
# load product clusters
clusters_df = pd.read_csv('/home/jk/data/online_retail_II_product_clusters.csv')
clusters = clusters_df['0'].to_list()

# load cleaned retail df
ret_cleaned = pd.read_csv('/home/jk/repos/predictive-segmentation/data/online_retail_II_w-cleaned.csv')
descriptions = ret_cleaned['Description'].unique()

# load list_products that specifies counts of keywords across product descriptions
with open('/home/jk/data/online_retail_II_list_products.csv', 'r') as read_obj:  
    csv_read = csv.reader(read_obj)  
    list_products = list(csv_read)

In the cleaned dataset, include **cluster** membership for each product, under the column **category_prod**

In [25]:
corresp = dict()
for key, val in zip(descriptions, clusters):
    corresp[key] = val 
# # ###
ret_cleaned['category_prod'] = ret_cleaned.loc[:, 'Description'].map(corresp)

In the next step, I create variables name **category_N** (with $N \in [0:4]$), indicating the amount of money spent for each category

In [26]:
for i in range(5):
    col = 'category_{}'.format(i)        
    df_temp = ret_cleaned[ret_cleaned['category_prod'] == i]
    price_temp = df_temp['Price'] * (df_temp['Quantity'] - df_temp['QuantityCanceled'])
    price_temp = price_temp.apply(lambda x:x if x > 0 else 0)
    ret_cleaned.loc[:, col] = price_temp
    ret_cleaned[col].fillna(0, inplace = True)
#__________________________________________________________________________________________________
ret_cleaned[['Invoice', 'Description', 'category_prod', 'category_0', 'category_1', 'category_2', 'category_3','category_4']][:5]

,Invoice,Description,category_prod,category_0,category_1,category_2,category_3,category_4
0,489434,15CM CHRISTMAS GLASS BALL 20 LIGHTS,3,0.0,0.0,0.0,83.4,0.0
1,489434,PINK CHERRY LIGHTS,3,0.0,0.0,0.0,81.0,0.0
2,489434,WHITE CHERRY LIGHTS,3,0.0,0.0,0.0,81.0,0.0
3,489434,"RECORD FRAME 7"" SINGLE SIZE",4,0.0,0.0,0.0,0.0,100.8
4,489434,STRAWBERRY CERAMIC TRINKET BOX,1,0.0,30.0,0.0,0.0,0.0


Until now the retail dataframes have been organized with one row for each product of every order transaction. Below I group products by transaction order (one row for each order), making a baskets dataframe that also indicates the category composition for each basket order.

In [34]:
#___________________________________________
# sum of purchases by customer and order
temp_df = ret_cleaned.groupby(by=['Customer ID', 'Invoice'], as_index=False)['TotalPrice'].sum()
baskets = temp_df.rename(columns = {'TotalPrice':'BasketPrice'})
#____________________________________________________________
# percentage of order price by product category
for i in range(5):
    col = 'category_{}'.format(i) 
    temp = ret_cleaned.groupby(by=['Customer ID', 'Invoice'], as_index=False)[col].sum()
    baskets.loc[:, col] = temp[col]
#_____________________
# order date
ret_cleaned['InvoiceDate_int'] = pd.to_datetime(ret_cleaned['InvoiceDate']).astype(int)
temp_date = ret_cleaned.groupby(by=['Customer ID', 'Invoice'], as_index=False)['InvoiceDate_int'].mean()
ret_cleaned.drop('InvoiceDate_int', axis = 1, inplace = True)
baskets.loc[:, 'InvoiceDate'] = pd.to_datetime(temp_date['InvoiceDate_int'])
#______________________________________
# select baskets with completed orders:
baskets = baskets[baskets['BasketPrice'] > 0]
baskets.sort_values('Customer ID', ascending = True)[:5]

,Customer ID,Invoice,BasketPrice,category_0,category_1,category_2,category_3,category_4,InvoiceDate
0,12346,491725,45.0,0.0,0.0,45.0,0.0,0.0,2009-12-14 08:34:00
1,12346,491742,22.5,0.0,0.0,22.5,0.0,0.0,2009-12-14 11:00:00
2,12346,491744,22.5,0.0,0.0,22.5,0.0,0.0,2009-12-14 11:02:00
3,12346,492718,22.5,0.0,0.0,22.5,0.0,0.0,2009-12-18 10:47:00
4,12346,492722,1.0,0.0,0.0,1.0,0.0,0.0,2009-12-18 10:55:00


In [36]:
baskets[8:15]

,Customer ID,Invoice,BasketPrice,category_0,category_1,category_2,category_3,category_4,InvoiceDate
8,12346,495295,18.00,0.00,0.00,18.00,0.00,0.00,2010-01-22 13:30:00
9,12346,499763,27.05,0.00,0.00,3.25,23.80,0.00,2010-03-02 13:08:00
10,12346,513774,142.31,0.00,0.00,0.00,142.31,0.00,2010-06-28 13:53:00
12,12347,529924,611.53,177.58,148.50,0.00,0.00,285.45,2010-10-31 14:20:00
13,12347,537626,711.79,23.40,187.20,276.35,141.44,83.40,2010-12-07 14:57:00
14,12348,524140,222.16,151.56,69.60,0.00,0.00,1.00,2010-09-27 14:59:00
15,12349,506394,1068.52,80.40,322.98,69.35,238.49,357.30,2010-04-29 13:20:00


In [38]:
print('{0} -> {1}'.format(baskets['InvoiceDate'].min(),  baskets['InvoiceDate'].max()))

2009-12-01 07:45:00 -> 2010-12-09 20:01:00
